## ReMKiT1D input generator - Gaussian advection with Matrix-free discretization

This notebook revisits the ReMKiT1D_advection_test notebook, instead using the `"shift"` procedure of `UnaryTransform` alongside the `Node` and `treeDerivation` objects to define the Divergence and Gradient operators without the use of Matrix or Stencil terms. It demonstrates:
- Central difference discretization at the Python level.
- (new in v1.2.0) Grid helper functions for obtaining dual cell widths and cell volumes.
- (new in v1.2.0) Support for the SUNDIALS `CVODE` integrator. Basic functionality is provided in this example.

This potentially allows for flexible top-level construction of matrices and boundary conditions without needing to be defined in the low-level Fortran library.

This notebook corresponds to Sections 5.2. and 6.1.1. of the ReMKiT1D code paper.

In [133]:
import sys
sys.path.append('../')
from RMK_support import RKWrapper ,Grid ,Node ,treeDerivation ,UnaryTransform
import RMK_support.simple_containers as sc
import RMK_support.IO_support as io

import numpy as np
import holoviews as hv
import matplotlib.pyplot as plt

### Wrapper initialization


In [134]:
rk = RKWrapper()

### Global parameters for writing the files


In [135]:
rk.jsonFilepath = "./config.json" # Default value
hdf5Filepath = "./RMKOutput/RMK_advection_matrix_free/"
rk.setHDF5Path(hdf5Filepath) # The input and output location of any HDF5 files used/generated by the code

### Setting options for external libraries used by ReMKiT1D


#### MPI


In [136]:
numProcsX = 4 # Number of processes in x direction
numProcsH = 1 # Number of processes in harmonic direction 
haloWidth = 1 # Halo width in cells 
numProcs = numProcsH * numProcsX
rk.setMPIData(numProcsX,numProcsH,haloWidth)

### Normalization


In [137]:
rk.setNormDensity(1.0e19) #n_0
rk.setNormTemperature(10.0) #T_0
rk.setNormRefZ(1.0) # reference ion charge for e-i collision time

### Grid initialization


In [138]:
# In normalized length or in meters - defaults to normalized unless isLengthInMeters=True in Grid
uniformGridWidth = 0.025
xGridWidths = uniformGridWidth*np.ones(512)
# In normalized velocity - default normalization is thermal velocity sqrt(m_e * k * T_e/2)
vGrid = np.ones(1) 
lMax = 0
gridObj = Grid(xGridWidths, vGrid, lMax, interpretXGridAsWidths=True)

rk.grid = gridObj

### Variables
Physical variables including density, temperature and flow are defined as before:


In [139]:
n = 1 + np.exp(-(gridObj.xGrid-np.mean(gridObj.xGrid))**2) # A Gaussian perturbation
rk.addVarAndDual('n',n,isCommunicated=True)

T = np.ones(len(gridObj.xGrid)) # Constant temperature
rk.addVar('T',T,isDerived=True) # isDerived removes the variable from the implicit vector
rk.addVarAndDual('u',isCommunicated=True,primaryOnDualGrid=True) # primaryOnDualGrid denotes that the main variable is u_dual, and u is interpolated 

# Note: In v1.2.0, the "time" scalar is added automatically unless the wrapper is instructed otherwise.

In [140]:
nNode = Node('n')
uNode = Node('u')
TNode = Node('T')

massRatio = 1/1836

wNode = 1.5*nNode*TNode + uNode**2/(nNode*massRatio) # assuming normalization to n_0*e*T_0

In [141]:
rk.addCustomDerivation("wDeriv",treeDerivation(wNode)) # Registering the derivation in the wrapper

In [142]:
rk.addVar("W",isDerived=True,derivationRule=sc.derivationRule("wDeriv",['n','u','T'])) # the derivation rule states which variables W depends on

Alongside the physical variables, we define stationary grid variables containing deep copies of data belonging to `rk.grid`:

In [143]:
# Grid spacing:
dx = rk.grid.xWidths
rk.addVar("dx",dx,isStationary=True,isCommunicated=True)

# Calculate inverse grid spacings for the finite difference scheme.
# This allows us to (safely) set 1/dx = 0 for points outside the grid.

# Inverse grid spacing of the left point (i-1):
dx_invL = 1/dx
dx_invL[0] = 0.
rk.addVar("dx_invL",dx_invL,isStationary=True,isCommunicated=True)

# Inverse grid spacing of the right point (i+1):
dx_invR = 1/dx
dx_invR[-1] = 0.
rk.addVar("dx_invR",dx_invR,isStationary=True,isCommunicated=True)

### Models

As before, the advection equation is given by:

$\frac{\partial n}{\partial t} = - \frac{\partial u}{\partial x}$

Instead of defining a Matrix and its stencil explicitly, we instead recreate the divergence stencil based on Eq.(15) of the ReMKiT1D paper:

$\nabla\cdot u = \frac{J_{i+\frac{1}{2}} \cdot u_{i+\frac{1}{2}} - J_{i-\frac{1}{2}} \cdot u_{i-\frac{1}{2}}}{J_i dx_i}$

...this time using `Node` and `UnaryTransform("shift")` objects to fetch values in $J$, $dx$ and $u$ from adjacent cells of a given grid.

Computationally, $u_{i-\frac{1}{2}}$ is equivalent to the $i$-th value in `Node('u_dual')`, while $u_{i+\frac{1}{2}}$ is found by shifting forward by one cell:

In [144]:
newModel = sc.CustomModel("nAdvection")

# Define the central difference divergence stencil:
def divNode(var, customNormConst, dx_invL, dx_invR):
    shiftFwd = UnaryTransform("shift",intParams=[+1])
    return customNormConst*(dx_invR*var - dx_invL*shiftFwd(var))

# Remember to pass customNormConst = -1:
# TODO: Pass in J.u rather than u alone.
rk.addCustomDerivation("divNode",derivOptions=treeDerivation(divNode(Node('u_dual'),-1.,Node('dx_invL'),Node('dx_invR'))))

newModel.addTerm("divFluxTerm",sc.DerivationTerm('n',sc.derivationRule("divNode",['u_dual','dx_invL','dx_invR'])))

rk.addModel(newModel) # Note: No need to specify newModel.dict() in v1.2.0+

Checking terms in model nAdvection:
   Checking term divFluxTerm


Likewise, the pressure gradient operator is given by:

$m_i \frac{\partial u}{\partial t} = - \frac{\partial (nkT)}{\partial x}$

The gradient stencil based on Eq.(16) of the ReMKiT1D paper:

$\nabla u = \frac{u_{i+\frac{1}{2}} - J_{i-\frac{1}{2}} \cdot u_{i-\frac{1}{2}}}{dx_i}$

...this time using `Node` and `UnaryTransform("shift")` objects to fetch values in $J$, $dx$ and $u$ from adjacent cells of a given grid.

For $dx_{i-\frac{1}{2}}$, we use the `xDualWidth`. TODO

For $n_{i-1}$, we shift backward by one cell:

In [145]:
newModel = sc.CustomModel("pGrad")

def pGradNode(var, customNormConst, dx_invL, dx_invR):
    shiftBwd = UnaryTransform("shift",intParams=[-1])
    return customNormConst*(shiftBwd(var) -var)/uniformGridWidth
    # TODO: Pass in (dx(i)+dx(i+1))/2 instead of uniformGridWidth

# Remember to pass the normalization constant, -m_e/2m_i, to ensure m_e*v**2/2 = k*T_0:
rk.addCustomDerivation("pGradNode",derivOptions=treeDerivation(pGradNode(Node('n'),-massRatio/2,Node('dx_invL'),Node('dx_invR'))))

newModel.addTerm("pGradTerm",sc.DerivationTerm('u_dual',sc.derivationRule("pGradNode",['n','dx_invL','dx_invR'])))

rk.addModel(newModel)

Checking terms in model pGrad:
   Checking term pGradTerm


### Integrator options

Available options include `BE`, `CVODE` and `RK`. Ideally, there should be little difference in the result if reasonable integrator options are used.

In [146]:
integratorOption = "RK"

# TODO: "BE" method causes segmentation fault during first timestep.

In [147]:
if integratorOption=="BE":
    rk.addIntegrator("BE",sc.picardBDEIntegrator(nonlinTol=1e-12,absTol=10.0,convergenceVars=['n','u_dual']))

elif integratorOption=="CVODE":
    rk.addIntegrator("CVODE",sc.CVODEIntegrator(absTol=1e-12))

elif integratorOption=="RK":
    rk.addIntegrator("RK",sc.rkIntegrator(3))

else:
    print('ERROR - Valid values for integratorOption = "BE", "CVODE", "RK"')


# Note: In v1.2.0, no need to specify active numbers of active groups in rk.setIntegratorGlobalData:
rk.setIntegratorGlobalData(initialTimestep=0.1) # in normalized time units

integratorStep = sc.IntegrationStep(integratorOption)

# Add active model groups to an integrator step with the following:
for tag in rk.modelTags(integrableOnly=True):
    integratorStep.addModel(tag,rk.activeGroups(tag),rk.activeGroups(tag))

rk.addIntegrationStep("Step"+integratorOption,integratorStep.dict())

In [148]:
rk.setFixedNumTimesteps(10000)
rk.setFixedStepOutput(200)

### Create config file

In [149]:
rk.writeConfigFile()

### Set global plotting options

In [ ]:
hv.extension('matplotlib')
%matplotlib inline 
plt.rcParams['figure.dpi'] = 150
hv.output(size=150,dpi=150)

### Load data from ReMKiT1D output files

In [ ]:
numFiles = 50
loadpath = hdf5Filepath
loadFilenames = [loadpath+f'ReMKiT1DVarOutput_{i}.h5' for i in range(numFiles+1)]
loadedData = io.loadFromHDF5(rk.varCont,filepaths=loadFilenames)
loadedData

In [ ]:
import panel as pn 
import RMK_support.dashboard_support as ds

pn.extension(comms="vscode") # change comms if not using VSCode
dashboard = ds.ReMKiT1DDashboard(loadedData,rk.grid)

dashboard.fluid2Comparison().show() # Removing .show() should display the dashboard inline - this can be buggy in some situations


### Compare with analytic solution

In [ ]:
wave_speed= np.sqrt(massRatio/2)
n_analytic=np.zeros((numFiles+1,gridObj.numX()))
times = loadedData.coords['time'].data
L = sum(xGridWidths)
for i in range(numFiles+1):
        leftPositionMod = (gridObj.xGrid-wave_speed*times[i]) % L
        leftPosition = np.where(leftPositionMod > 0,leftPositionMod,leftPositionMod+L)
        rightPosition = (gridObj.xGrid+wave_speed*times[i]) % L
        n_analytic[i,:] =1 + 0.5*(np.exp(-(leftPosition-np.mean(gridObj.xGrid))**2) + np.exp(-(rightPosition-np.mean(gridObj.xGrid))**2)) 


In [ ]:
dataName = 'n'

curveDict = {t: hv.Scatter(loadedData[dataName][{"time":t}],label='simulation').opts(marker="o",color="r",s=6.0)*hv.Curve((gridObj.xGrid,n_analytic[t,:]),label='analytic result').opts(title=f't = {loadedData["time"].values[t]:.2f} '+loadedData.coords["time"].attrs["units"],fontscale=2, fig_size=150,linewidth=3.0) for t in range(numFiles+1)}
kdims = [hv.Dimension(('time', 'Time'),unit=loadedData.coords["time"].attrs["units"], default=0)]
hv.HoloMap(curveDict,kdims=kdims).opts()

### Check if W is calculated correctly

In [ ]:
dataName = 'W'

testWCalc = loadedData['n']*loadedData['T'] * 1.5 + loadedData['u']**2 /( loadedData['n']*massRatio) - loadedData['W']

print(testWCalc.where(np.abs(testWCalc)>5e-16,drop=True))

### Producing graphs for the paper

### Relative error wrt analytic solution

In [ ]:
diff = np.abs(n_analytic - loadedData['n'])/n_analytic

In [ ]:
relativeErrorPlot=hv.Curve(diff.reduce(np.max,'x')).opts(ylabel='$\delta n$',marker='o',fontscale=2, fig_size=150,linewidth=3.0)

In [ ]:
relativeErrorPlot.opts()

In [ ]:
hv.output(fig='pdf')
hv.save(relativeErrorPlot, 'advectionTestRelErr.pdf', dpi=144)

### Final simulation state

In [ ]:
hv.save(curveDict[50].opts(legend_position='top',legend_cols=1,title=''),'finalDensityAdv.pdf',dpi=144)